In [1]:
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## Carregando a base de dados

In [36]:
data = pd.read_csv('loan.csv')

In [37]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [39]:
data.Loan_Status.value_counts()

Loan_Status
Y    422
N    192
Name: count, dtype: int64

In [40]:
data2 = data[data.Loan_Status=='Y'].sample(200)

In [41]:
data = pd.concat([data2, data[data.Loan_Status == 'N'].sample(192)])

In [42]:
data.Loan_Status.value_counts()

Loan_Status
Y    200
N    192
Name: count, dtype: int64

## Checando Missing Values

In [43]:
data2.isnull().sum()

Loan_ID               0
Gender                6
Married               2
Dependents            3
Education             0
Self_Employed        10
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      4
Credit_History       16
Property_Area         0
Loan_Status           0
dtype: int64

Preechendo Missing Values:

- `Dependents`: Assumindo o valor majoritário da coluna.
- `Self_Employed`: Assumindo o valor majoritário da coluna.
- `Loan_Amount_Term`: Preenchendo com o valor médio da coluna.
- `Credit_History`: Assumindo o valor majoritário da coluna.
- `Married`: Assumindo o valor majoritário da coluna.
- `Gender`: Assumindo o valor majoritário da coluna.

In [44]:
data['Gender'] = data['Gender'].fillna('Male')

In [45]:
data['Married'] = data['Married'].fillna('No')

In [46]:
data['Dependents'] = data['Dependents'].fillna('0')

In [47]:
data['Self_Employed'] = data['Self_Employed'].fillna('No')

In [48]:
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mean())

In [49]:
data['Credit_History'] = data['Credit_History'].fillna(1.0)

In [50]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean())

In [51]:
data.Credit_History.value_counts()

Credit_History
1.0    305
0.0     87
Name: count, dtype: int64

### Checando novamente Missing Values

In [52]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

**Transformando dados categóricos**

Várias colunas do dataframe são categóricas, precisamos transforma-las, são elas: `Gender`, `Married`, `Education`, `Self_Employed` & `Property_Area` columns.

In [53]:
from sklearn.preprocessing import LabelEncoder

In [54]:
gender_values = {'Female' : 0, 'Male' : 1} 
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
loan_values = {'Y':1,'N':0}
data.replace({'Gender': gender_values,
                 'Married': married_values, 
                 'Education': education_values,
                 'Self_Employed': employed_values, 
                 'Dependents': dependent_values,
                 'Loan_Status': loan_values
                }, inplace=True)

In [55]:
data.drop(['Loan_ID','CoapplicantIncome','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)

In [56]:
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Status
11,1,1,2,0,0,2500,109.0,1
37,0,1,0,0,0,3667,144.0,1
422,1,1,0,0,0,1820,100.0,1
229,1,0,0,0,1,6400,200.0,1
182,1,1,0,0,0,4600,73.0,1


Selecionando o melhor classificador através de Pipeline e GridSearchCV

In [57]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib  # Corrected import
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [58]:
pipe_random_forest = Pipeline([
        ('scl', StandardScaler()),
        
        ('clf', RandomForestClassifier())
        ]
)

In [59]:
pipe_svm = Pipeline([
        ('scl', StandardScaler()),
        ('clf', svm.SVC())
        ]
)

In [60]:
pipe_knn = Pipeline([
        ('scl', StandardScaler()),
        ('clf', KNeighborsClassifier())
        ]
)

#### Valores para Grid

In [61]:
valores = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [62]:
grid_params_rf = [{
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': valores,
    'clf__max_depth': valores,
    'clf__min_samples_split': valores[1:]
}]

In [63]:
grid_params_svm = [{
    'clf__kernel': ['linear', 'rbf'], 
    'clf__C': valores
}]

In [64]:
grid_params_knn = [{
    'clf__n_neighbors': valores,
}]

#### Construindo GridSearch

In [65]:
gs_rf = GridSearchCV(
    estimator=pipe_random_forest,
    param_grid=grid_params_rf,
    scoring='accuracy',
    cv=10 
)

In [66]:
gs_svm = GridSearchCV(
    estimator=pipe_svm,
    param_grid=grid_params_svm,
    scoring='accuracy',
    cv=10,
)

In [67]:
gs_knn = GridSearchCV(
    estimator=pipe_knn,
    param_grid=grid_params_knn,
    scoring='accuracy',
    cv=10,
)

In [68]:
X_train = data.drop('Loan_Status',axis=1)
y = data['Loan_Status']

#### Computando o GridSearch para Random Forest

In [69]:
gs_rf.fit(X_train,y)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', RandomForestClassifier())]),
             param_grid=[{'clf__criterion': ['gini', 'entropy'],
                          'clf__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'clf__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                    10],
                          'clf__min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9,
                                                     10]}],
             scoring='accuracy')

#### Melhores parametros e scoring

In [70]:
print('Melhores parâmetros: %s' % gs_rf.best_params_)
print('Melhores Acurácia: %.3f' % gs_rf.best_score_)

Melhores parâmetros: {'clf__criterion': 'entropy', 'clf__max_depth': 3, 'clf__min_samples_leaf': 10, 'clf__min_samples_split': 9}
Melhores Acurácia: 0.567


#### Computando o GridSearch para SVM

In [71]:
gs_svm.fit(X_train,y)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', SVC())]),
             param_grid=[{'clf__C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'clf__kernel': ['linear', 'rbf']}],
             scoring='accuracy')

#### Melhores parametros e scoring

In [72]:
print('Melhores parâmetros: %s' % gs_svm.best_params_)
print('Melhores Acurácia: %.3f' % gs_svm.best_score_)

Melhores parâmetros: {'clf__C': 8, 'clf__kernel': 'rbf'}
Melhores Acurácia: 0.543


#### Computando o GridSearch para KNN

In [73]:
gs_knn.fit(X_train,y)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', KNeighborsClassifier())]),
             param_grid=[{'clf__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}],
             scoring='accuracy')

#### Melhores parametros e scoring

In [74]:
print('Melhores parâmetros: %s' % gs_knn.best_params_)
print('Melhores Acurácia: %.3f' % gs_knn.best_score_)

Melhores parâmetros: {'clf__n_neighbors': 10}
Melhores Acurácia: 0.548


### Métricas de Validação ###

In [75]:
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [76]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X_train, y)

In [77]:
print (pd.crosstab(y_teste, gs_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

Predito   0   1  All
Real                
0        25  19   44
1        19  35   54
All      44  54   98 


In [78]:
print (metrics.classification_report(y_teste,gs_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.57      0.57      0.57        44
           1       0.65      0.65      0.65        54

    accuracy                           0.61        98
   macro avg       0.61      0.61      0.61        98
weighted avg       0.61      0.61      0.61        98



### Persistindo o modelo de Machine Learning para o disco. ###

In [80]:
import joblib

#### Persistindo o melhor modelo em disco.

In [81]:
joblib.dump(gs_rf, 'model.pkl')

['model.pkl']

#### Listando os arquivos em disco.

In [82]:
!ls

array.npy
lista.pkl
lista2.npy
lista3.gz
lista3.joblib
loan.csv
model.pkl
persistencia-objetos-disco.ipynb
persistindo-modelo-machine-learning-disco.ipynb
requirements.txt


#### Carregando o modelo a partir do disco para a memória.

In [83]:
model = joblib.load('model.pkl')

In [84]:
print("Atributos do Modelo:\n\nClasses:{}\nEstimator:{}".format(model.classes_,model.estimator))

Atributos do Modelo:

Classes:[0 1]
Estimator:Pipeline(steps=[('scl', StandardScaler()), ('clf', RandomForestClassifier())])


**Verificando o Dataset final gerado.**

In [85]:
X_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount
11,1,1,2,0,0,2500,109.0
37,0,1,0,0,0,3667,144.0
422,1,1,0,0,0,1820,100.0
229,1,0,0,0,1,6400,200.0
182,1,1,0,0,0,4600,73.0


**Teste de Classificação.**

In [86]:
teste = np.array([[1,1,3,0,0,9504,275.0]])

In [87]:
model.predict(teste)

array([1])

**Probabilidades de Classes.**

In [88]:
model.predict_proba(teste)

array([[0.40281611, 0.59718389]])